In [1]:
import os
import copy
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from pandarallel import pandarallel
import numpy as np
import matplotlib.pyplot as plt

from microsim.population import NHANESDirectSamplePopulation
from microsim.statsmodel_linear_risk_factor_model import StatsModelLinearRiskFactorModel
from microsim.regression_model import RegressionModel
from microsim.data_loader import load_model_spec
from microsim.outcome import OutcomeType, Outcome
from microsim.outcome_details.stroke_details import StrokeSubtypeModelRepository, StrokeNihssModel, StrokeTypeModel
from microsim.stroke_outcome import StrokeOutcome, StrokeSubtype, StrokeType, Localization
from microsim.treatment_strategy_repository import TreatmentStrategyRepository
from microsim.bp_treatment_strategies import AddASingleBPMedTreatmentStrategy, AddNBPMedsTreatmentStrategy
from microsim.treatment import TreatmentStrategiesType, TreatmentStrategyStatus
from microsim.outcome import OutcomeType
from microsim.stroke_partition_model import StrokePartitionModel

import scipy.special as scipySpecial

microsimDir = "/Users/deligkaris.1/OneDrive - The Ohio State University Wexner Medical Center/MICROSIM/CODE/microsim"
os.chdir(microsimDir)

import time

timeI = time.time()

popSize = 600000
nYears = 5
bpMedsAdded = 1

In [2]:
treatmentStrategies = TreatmentStrategyRepository()
#treatmentStrategies._repository[TreatmentStrategiesType.BP.value] = AddASingleBPMedTreatmentStrategy()
treatmentStrategies._repository[TreatmentStrategiesType.BP.value] = AddNBPMedsTreatmentStrategy(bpMedsAdded)

In [3]:
%%time
controlPop = NHANESDirectSamplePopulation(popSize, 1999)

CPU times: user 3min 59s, sys: 1.7 s, total: 4min
Wall time: 4min 1s


In [4]:
%%time
controlPop.advance_parallel(nYears, None, 5)

CPU times: user 1min 58s, sys: 2min 17s, total: 4min 16s
Wall time: 17min 33s


In [5]:
%%time
treatmentPop = NHANESDirectSamplePopulation(popSize, 1999)

CPU times: user 4min 30s, sys: 14.5 s, total: 4min 45s
Wall time: 4min 48s


In [6]:
%%time
treatmentPop.advance_parallel(1, treatmentStrategies = treatmentStrategies, nWorkers=5)
treatmentStrategies._repository[TreatmentStrategiesType.BP.value].status = TreatmentStrategyStatus.MAINTAIN
treatmentPop.advance_parallel(nYears-1, treatmentStrategies = treatmentStrategies, nWorkers=5)

CPU times: user 5min 44s, sys: 22min 3s, total: 27min 48s
Wall time: 56min


In [7]:
controlRisk = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.STROKE), 
                           controlPop._people)))/controlPop._n
treatmentRisk = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.STROKE), 
                             treatmentPop._people)))/treatmentPop._n
strokeRR = treatmentRisk/controlRisk

controlRiskMI = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.MI), 
                           controlPop._people)))/controlPop._n
treatmentRiskMI = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.MI), 
                             treatmentPop._people)))/treatmentPop._n
miRR = treatmentRiskMI/controlRiskMI

In [8]:
strokeRR, miRR

(0.7902542372881356, 0.8606392938344256)

In [9]:
from microsim.cv_model import CVModelMale, CVModelFemale
import importlib.util
moduleCV = importlib.import_module("microsim.cv_model_repository")
moduleStroke = importlib.import_module("microsim.stroke_partition_model_repository")

In [10]:
##19 -11.8284175 -12.9715475 0.00390625 -2.4164418087083006 0.001953125 0.7823834196891192 0.8673245614035087

#interceptStep = 0.5
interceptStep = 0.01
interceptChange = 0
#interceptMale = -11.679980
interceptMale = -11.7959175
#interceptFemale = -12.823110
interceptFemale = -12.9390475

#strokeInterceptStep = 0.25
strokeInterceptStep = 0.01
strokeInterceptChange = 0
model_spec = load_model_spec("StrokeMIPartitionModel")
#strokeIntercept = model_spec["coefficients"]["Intercept"]
strokeIntercept = -2.4001918

strokeRRRef = 0.79**bpMedsAdded
miRRRef = 0.87**bpMedsAdded

In [13]:
%%time
for i in range(10):

    strokeRRDiff = strokeRR-strokeRRRef
    miRRDiff = miRR-miRRRef
    
    if (strokeRRDiff>0) & (miRRDiff>0):
        interceptChange = -interceptStep
    elif (strokeRRDiff<0) & (miRRDiff<0):
        interceptChange = interceptStep
    else:
        interceptChange = 0
    interceptMale += interceptChange
    interceptFemale += interceptChange

    if strokeRRDiff>0:
        strokeInterceptChange = -strokeInterceptStep
    else:
        strokeInterceptChange = strokeInterceptStep
    strokeIntercept += strokeInterceptChange
    
    print(i, interceptMale, interceptFemale, interceptChange, strokeIntercept, strokeInterceptChange, 
              strokeRR, miRR)    
    
    class CVModelMaleFor1bpMedsAdded(CVModelMale):
        def __init__(self):
            super().__init__(intercept = interceptMale)

    class CVModelFemaleFor1bpMedsAdded(CVModelFemale):
        def __init__(self):
            super().__init__(intercept = interceptFemale)

    setattr(moduleCV, "CVModelFemaleFor1bpMedsAdded", CVModelFemaleFor1bpMedsAdded)
    setattr(moduleCV, "CVModelMaleFor1bpMedsAdded", CVModelMaleFor1bpMedsAdded)
    
    class StrokePartitionModelFor1bpMedsAdded(StrokePartitionModel):
        def __init__(self):
            intercept = strokeIntercept
            super().__init__(intercept=intercept)
            
    setattr(moduleStroke, "StrokePartitionModelFor1bpMedsAdded", StrokePartitionModelFor1bpMedsAdded)
    
    treatmentPop = NHANESDirectSamplePopulation(popSize, 1999)
    treatmentStrategies._repository[TreatmentStrategiesType.BP.value].status = TreatmentStrategyStatus.BEGIN
    treatmentPop.advance(1, treatmentStrategies = treatmentStrategies)
    treatmentStrategies._repository[TreatmentStrategiesType.BP.value].status = TreatmentStrategyStatus.MAINTAIN
    treatmentPop.advance(nYears-1, treatmentStrategies = treatmentStrategies)
     
    treatmentRisk = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.STROKE), 
                             treatmentPop._people)))/treatmentPop._n
    strokeRR = treatmentRisk/controlRisk
    
    treatmentRiskMI = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.MI), 
                                 treatmentPop._people)))/treatmentPop._n
    miRR = treatmentRiskMI/controlRiskMI
    
    if ((strokeRR-strokeRRRef)*strokeRRDiff<0) & ((miRR-miRRRef)*miRRDiff<0):
        interceptStep = interceptStep/2
        strokeInterceptStep = strokeInterceptStep/2

0 -11.790917499999997 -12.934047499999997 0 -2.4295667999999986 -0.000625 0.8042372881355934 0.8670589808746825
1 -11.790292499999998 -12.933422499999997 0.000625 -2.4289417999999987 0.000625 0.7773305084745763 0.8507422763140298
2 -11.790292499999998 -12.933422499999997 0 -2.4295667999999986 -0.000625 0.8004237288135594 0.8586331416343453
3 -11.790292499999998 -12.933422499999997 0 -2.4292542999999984 0.0003125 0.7745762711864408 0.8840444028353618
4 -11.790292499999998 -12.933422499999997 0 -2.4294105499999983 -0.00015625 0.7972457627118645 0.8621104721144845
5 -11.790292499999998 -12.933422499999997 0 -2.429566799999998 -0.00015625 0.7991525423728815 0.8407115153136285
6 -11.790136249999998 -12.933266249999997 0.00015625 -2.4294105499999983 0.00015625 0.7877118644067796 0.8610405242744417
7 -11.789979999999998 -12.933109999999997 0.00015625 -2.4292542999999984 0.00015625 0.7597457627118644 0.8457937675538318
8 -11.789823749999998 -12.932953749999998 0.00015625 -2.4290980499999986 0.

In [12]:
timeF = time.time()
(timeF-timeI)/60

723.9069179495176

In [ ]:
strokeRRDiff = strokeRR-strokeRRRef
if strokeRRDiff>0:
    interceptChange = -interceptStep
else:
    interceptChange= interceptStep
interceptMale += interceptChange
interceptFemale += interceptChange

print(interceptMale, interceptFemale, interceptStep, interceptChange, strokeRRDiff, strokeRR)

In [ ]:
#19 -11.8284175 -12.9715475 0.00390625 -2.4164418087083006 0.001953125 0.7823834196891192 0.8673245614035087
interceptMale = -11.8469721875
interceptFemale = -12.9901021875
interceptChange = -0.00048828125

In [ ]:
controlRiskMI = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.MI), 
                           controlPop._people)))/controlPop._n
treatmentRiskMI = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.MI), 
                             treatmentPop._people)))/treatmentPop._n
miRR = treatmentRiskMI/controlRiskMI
miRR

In [ ]:
#RESULTS
#1 bp: -11.795917500000009 -12.939047500000008 0 -2.400191808699997 -0.00125 0.7918781725888325 0.8565955117679255
#2 bp -11.931250000000002 -13.051250000000003 0 -2.6159375 0.0015625 0.6071428571428572 0.7655677655677655
#2 bp: 15 -11.911249999999999 -13.031249999999998 0 -2.5334375000000016 0.0025 0.6126154826377828 0.7658060581418988
#3 bp: -12.007714375000003 -13.150500000000003 -0.0125 -2.6649999999999987 -0.0125 0.5244482907832108 0.6721311475409835
#4 bp: -12.085210000000002 -13.228000000000002 0 -2.7462 0.01 0.38026981450252956 0.5751421608448416

In [ ]:
%%time
treatmentPop = NHANESDirectSamplePopulation(popSize, 1999)

In [ ]:
%%time
treatmentPop.advance(nYears, treatmentStrategies = treatmentStrategies)

In [ ]:
controlRisk = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.STROKE), 
                           controlPop._people)))/controlPop._n
treatmentRisk = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.STROKE), 
                             treatmentPop._people)))/treatmentPop._n
strokeRR = treatmentRisk/controlRisk
strokeRR

In [ ]:
if (strokeRR-strokeRRRef)*strokeRRDiff<0:
    interceptStep = interceptStep/2
strokeRRDiff = strokeRR-strokeRRRef

In [ ]:
test = {"male": {"1": "one",
                 "2": "two"},
        "female": {"1": "one",
                 "2": "two"}}

In [ ]:
test["male"]["2"]